In [1]:
import pandas as pd
import re
#set ups
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
# importing package 
import matplotlib.pyplot as plt 
import numpy as np 
import requests

#setups
%matplotlib inline


In [2]:
df=pd.read_csv("listings_full.csv")

## Doing all the checks needed

In [3]:
df.shape

(13182, 75)

In [4]:
df.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
source                                           object
name                                             object
description                                     float64
neighborhood_overview                            object
picture_url                                      object
host_id                                           int64
host_url                                         object
host_name                                        object
host_since                                       object
host_location                                    object
host_about                                       object
host_response_time                               object
host_response_rate                               object
host_acceptance_rate                            

In [5]:
# keeping the columns i am interested in
df=df[['id',
       'name', 
       'host_id',
       'host_name', 
       'host_since',
       'host_location', 
       'host_is_superhost',
       'host_neighbourhood', 
       'host_listings_count',
       'host_total_listings_count',
       'neighbourhood',
       'neighbourhood_cleansed',
       'latitude', 'longitude',
       'property_type',
       'room_type',
       'bathrooms_text',
       'beds',
       'price',
       'minimum_nights', 
       'maximum_nights',
       'minimum_minimum_nights',
       'maximum_minimum_nights',
       'minimum_maximum_nights',
       'maximum_maximum_nights',
       'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 
       'availability_365', 
       'review_scores_rating',
       'review_scores_accuracy',
       'license',
       'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms']].copy()

In [6]:
df.describe()

,id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,beds,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_365,review_scores_rating,review_scores_accuracy,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
count,1.318200e+04,1.318200e+04,13182.000000,13182.000000,13182.000000,13182.000000,13017.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,13182.000000,11075.000000,11075.000000,13182.000000,13182.000000,13182.000000,13182.000000
mean,3.954590e+17,2.166796e+08,20.644895,32.083447,37.980172,23.731728,2.071368,3.151267,571.466925,3.197390,4.595812,757.426567,781.464421,4.157988,767.141405,222.418146,4.737552,4.783382,12.034441,10.849416,1.031027,0.071082
std,4.270628e+17,1.732957e+08,61.423587,136.062916,0.013427,0.012480,1.412656,16.231489,457.941252,18.416376,26.587753,449.324853,437.633288,24.276247,440.906213,136.667965,0.431216,0.394101,19.431829,18.978576,4.439618,0.850407
min,1.059500e+04,1.805000e+03,1.000000,1.000000,37.950322,23.697700,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,3.166800e+07,6.022029e+07,1.000000,2.000000,37.969140,23.723970,1.000000,1.000000,90.000000,1.000000,1.000000,365.000000,365.000000,1.000000,365.000000,88.000000,4.680000,4.750000,1.000000,1.000000,0.000000,0.000000
50%,5.312285e+07,1.821563e+08,5.000000,6.000000,37.978573,23.729719,2.000000,2.000000,365.000000,2.000000,2.000000,1125.000000,1125.000000,2.000000,1125.000000,268.000000,4.860000,4.890000,4.000000,3.000000,0.000000,0.000000
75%,8.516889e+17,3.696750e+08,20.000000,23.000000,37.988298,23.737657,3.000000,2.000000,1125.000000,2.000000,3.000000,1125.000000,1125.000000,2.300000,1125.000000,348.000000,4.980000,5.000000,14.000000,12.000000,0.000000,0.000000
max,1.052831e+18,5.516695e+08,2444.000000,5529.000000,38.030830,23.780220,30.000000,800.000000,4444.000000,800.000000,1000.000000,4444.000000,4444.000000,800.000000,4444.000000,365.000000,5.000000,5.000000,121.000000,121.000000,44.000000,15.000000


In [7]:
# What room type is more common?
df['room_type'].value_counts(normalize=True)*100

room_type
Entire home/apt    90.320134
Private room        8.276438
Hotel room          0.773782
Shared room         0.629646
Name: proportion, dtype: float64

### Πόσοι host έχουν πάνω από δύο καταχωρήσεις;

Χρειαζόμαστε μόνο τις περιπτώσεις όπου ένας οικοδεσπότης παρέχει ένα ολόκληρο σπίτι. <br>Τα δωμάτια ξενοδοχείων δεν περιλαμβάνονται σε αυτό το μέρος της ανάλυσης, καθώς θεωρούμε ότι δεν συμμετέχουν στην πτυχή της πολλαπλής ιδιοκτησίας.

#### Ολόκληρο κατάλυμα
Τα ολόκληρα καταλύματα είναι ιδανικά αν θέλετε να νιώθετε σαν στο σπίτι σας. Στο ολόκληρο κατάλυμα, θα έχετε ολόκληρο τον χώρο διαθέσιμο μόνο για εσάς. Το ολόκληρο κατάλυμα περιλαμβάνει συνήθως ένα υπνοδωμάτιο, ένα μπάνιο, μια κουζίνα και μια ξεχωριστή, αποκλειστική είσοδο. Οι οικοδεσπότες θα πρέπει να σημειώσουν στην περιγραφή αν θα βρίσκονται στην ιδιοκτησία ή όχι (π.χ. "Ο οικοδεσπότης μένει στον πρώτο όροφο του καταλύματος") και να συμπεριλάβουν περισσότερες λεπτομέρειες για την καταχώρηση.


In [8]:
# κρατάμε τα ολόκληρα καταλύματα
ent_house=df[df.room_type.str.contains('Entire',case=False)].copy().reset_index()
# βρίσκω πόσα entire homes έχει ο κάθε host 
listing = pd.DataFrame(ent_house.host_id.value_counts(ascending=False)).copy().reset_index()

In [9]:
# κατηγοριοποιώ πόσα ακίνητα έχει ο κάθε host (οικοδεσπότης)
# με βάση την ελληνική νομοθεσία, από 3 και πάνω, η κατηγοριοποίηση αλλάζει
listing['many_properties']= listing['count'].apply(lambda x: 1 if x>2 else 0)

## Ποιο είναι το ποσοστό των οικοδεσποτών που έχουν περισσότερα από 2 ακίνητα;

In [ ]:
# how_many_porperties= listing['many_properties'].value_counts(normalize=True)*100\

# text= f' Στην πλατφόρμα του Airbnb, to {round(how_many_porperties[1])} των host(οικοδεσποτών) έχουν περισσότερα από 2 ακίνητα. 

## Ποιές περιοχές έχουν περισσότερες καταχωρήσεις(listings);

In [10]:
# keeping only the values that are true and using it to identify these hosts in the general dataset
list_business_hosts = listing[listing['many_properties']>0].host_id.tolist()

In [11]:
# περνάς τη νέα μεταβλητή στο df σου
ent_house['many_properties'] = ent_house['host_id'].apply(lambda x: 1 if x in list_business_hosts else 0)

### Περιοχές με  ποσοστό των καταχωρήσεων

In [12]:
ent_house['neighbourhood_cleansed'].value_counts(normalize=True)*100

neighbourhood_cleansed
ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ              17.772552
ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ                  8.945070
ΝΕΟΣ ΚΟΣΜΟΣ                          7.878381
ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ             7.492021
ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ     5.954981
ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ                      4.342348
ΠΑΓΚΡΑΤΙ                             4.233160
ΑΜΠΕΛΟΚΗΠΟΙ                          3.586427
ΚΟΛΩΝΑΚΙ                             3.309256
ΑΚΡΟΠΟΛΗ                             3.099278
ΠΕΤΡΑΛΩΝΑ                            2.998488
ΘΗΣΕΙΟ                               2.880900
ΚΕΡΑΜΕΙΚΟΣ                           2.628927
ΚΥΨΕΛΗ                               2.024189
ΣΤΑΔΙΟ                               1.931799
ΓΟΥΔΙ                                1.839409
ΙΛΙΣΙΑ                               1.705023
ΖΑΠΠΕΙΟ                              1.411053
ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ                     1.343860
ΝΙΡΒΑΝΑ                              1.285066
ΛΥΚΑΒΗΤΤΟΣ                           1.285066
ΠΑΤΗΣΙΑ    

### Ποιο είναι το ποσοστό των κατοικιών που ανήκουν σε επαγγελματίες οικοδεσπότες ;

In [13]:
kk=ent_house["many_properties"].value_counts(normalize=True)*100

In [38]:
# owned_by_prof=kk[1]
# owned_by_amateurs=kk[0]

# text= f'Από τις κατοικίες που έχουν καταχωρηθεί στην πλατφόρμα, το {round(owned_by_prof)}% ανήκει σε επαγγελματίες οικοδεσπότες \
# ένω το {round(owned_by_amateurs)}% ανήκει σε οικοδεσπότες χωρίς επαγγελματική δραστηριότητα.'

# text

### Καθαρίζουμε τις τιμές


In [18]:
from money_parser import price_str
from decimal import Decimal

In [19]:
def correct_prices(dirty_price):
    if not pd.isna(dirty_price): 
        amount_num = price_str(dirty_price) 
        amount_decimal = Decimal(str(amount_num)) 
        return float(amount_decimal) 

In [20]:
ent_house['cleaned_price'] = ent_house['price'].apply(lambda x: correct_prices(x))

## Mέση τιμή ανά διανυκτέρευση

### Για κάθε περιοχή ξεχωριστά

In [21]:
# ομαδοποίηση με βάση τις στήλες 'neighborhood_cleansed', 'many_properties' 
# και εφαρμόζουμε συγκεντρωτικά τις του μέσου όρου και της διάμεσου στις τιμές που καθαρίσαμε.

price_per_night_area=ent_house.groupby(by=['neighbourhood_cleansed', 'many_properties'])['cleaned_price'].agg(['median', 'mean']).reset_index()
price_per_night_area

,neighbourhood_cleansed,many_properties,median,mean
0,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,0,49.0,54.625000
1,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,1,93.0,122.875000
2,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,0,40.0,48.000000
3,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,1,73.0,69.909091
4,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,0,51.0,67.785479
5,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,1,46.0,59.519084
6,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,0,40.0,56.285714
7,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,1,50.0,54.857143
8,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,0,47.0,55.690909
9,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,1,46.0,44.555556


In [22]:
# κρατάμε τη μέση τιμή για τις περιοχές
ent_house.groupby(by=['neighbourhood_cleansed', 'many_properties'])['cleaned_price'].median().sort_values()

neighbourhood_cleansed            many_properties
ΚΟΛΟΚΥΝΘΟΥ                        0                   40.0
ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ                  0                   40.0
ΓΚΥΖΗ                             1                   40.0
ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ                    0                   40.0
ΝΙΡΒΑΝΑ                           0                   41.0
ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ                  1                   43.0
ΑΝΩ ΚΥΨΕΛΗ                        1                   43.0
ΠΕΝΤΑΓΩΝΟ                         0                   43.5
ΚΟΛΟΚΥΝΘΟΥ                        1                   44.5
ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ                   0                   45.0
ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ                  0                   45.0
ΑΝΩ ΚΥΨΕΛΗ                        0                   45.0
ΝΙΡΒΑΝΑ                           1                   45.0
ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ                 1                   46.0
ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ  1                   46.0
ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ                 0                   47.0
ΑΝΩ ΠΑ

### Για όλη την Αθήνα

In [23]:
price_per_night=ent_house.groupby(by=['many_properties'])['cleaned_price'].agg(['median', 'mean']).reset_index()

In [40]:
# amateur_hosts = price_per_night['median'][0]
# prof_hosts = price_per_night['median'][1]

# precentage_difference=((prof_hosts-amateur_hosts)/((prof_hosts+amateur_hosts)/2))*100

# text= f'Η μέση τιμή του διαμερίσματος ανά μέρα για τους οικοδεσπότες με 3 και πάνω ακίνητα είναι {prof_hosts} \
# και για όσους έχουν έχουν κάτω απο 3 είναι {amateur_hosts}. Οι τιμές αυτές έχουν διαφορά {round(precentage_difference)}%'
# text

## Ποιές είναι οι 5 περιοχές με τη μεγαλύτερη διαφορά τιμών;

In [24]:
# κρατάμε το median των τιμών ανά περιοχή
price_per_night_area_med= price_per_night_area[['neighbourhood_cleansed','many_properties','median']].reset_index().copy()
price_per_night_area_med

,index,neighbourhood_cleansed,many_properties,median
0,0,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,0,49.0
1,1,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,1,93.0
2,2,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,0,40.0
3,3,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,1,73.0
4,4,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,0,51.0
5,5,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,1,46.0
6,6,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,0,40.0
7,7,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,1,50.0
8,8,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,0,47.0
9,9,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,1,46.0


In [25]:
# ξαναστείνουμε το df ώστε να υπάρχουνε σε μία στηλη οι τιμές όσων έχουν λιγότερα από 3 διαμερίσματα και σε μία άλλη όσων εχουν περισσότερα
pay_comp=price_per_night_area_med.pivot_table(values='median', index='neighbourhood_cleansed', columns='many_properties').reset_index().copy()
pay_comp=pay_comp.rename(columns={0:'less_than_3', 
                                  1:'more_than_3'})

In [26]:
#  βρίσκουμε τη διαφορά των τιμών ανά περιοχή και στήνουμε το df ώστε να φαίνονται πρώτες οι περιοχές με τη μεγαλύτερη διαφορά
pay_comp['price_difference']= (pay_comp.more_than_3 - pay_comp.less_than_3 )
pay_comp = pay_comp.sort_values(by='price_difference', ascending=False).reset_index(drop=True).copy()

In [39]:
# areas_greatest_price_dif= ', '.join(pay_comp['neighbourhood_cleansed'].loc[:4].tolist())

# text= f'Οι 5 γειτονιές με τη μεγαλύτερη διαφορά τιμών μεταξύ των επαγγελματιών host\
# και των ερασιτεχνών είναι οι {areas_greatest_price_dif}'
# text

### Σε πόσες περιοχές τα διαμέρίσματα που ανήκουν σε επαγγελματίες οικοδεσπότες είναι πιο ακριβά;

In [27]:
more=[]
less=[]

for i, item in pay_comp.iterrows():
    if item['price_difference'] > 0:
        more.append(item['neighbourhood_cleansed'])
    elif item['price_difference'] < 0:
        less.append(item['neighbourhood_cleansed'])

In [41]:
# more_than_0= len(more)
# less_than_0= len(less)

# text= f'Σε {more_than_0} περιοχές της Αθήνας, τα διαμερίσματα που ανήκουν σε επαγγελματίες hosts είναι πιο ακριβά από \
# ότι τα διαμερίσματα που ανήκουν σε ερασιτέχνες hosts που είναι ακριβότερα σε {less_than_0} περιοχές.'
# text

In [28]:
#  βρίσκουμε την διαφορά για κάθε περιοχή
pay_comp['percentage_difference'] = ((pay_comp['more_than_3'] - pay_comp['less_than_3']) / ((pay_comp['more_than_3'] + pay_comp['less_than_3']) / 2)) * 100
pay_comp=pay_comp.sort_values(by='percentage_difference', ascending=False)

In [42]:
# areas_greatest_price_dif= ', '.join(pay_comp['neighbourhood_cleansed'].loc[:4].tolist())

# text= f'Οι 5 γειτονιές με τη μεγαλύτερη διαφορά τιμών μεταξύ των επαγγελματιών host\
# και των ερασιτεχνών είναι οι {areas_greatest_price_dif}. Η Μεγαλύτερη διαφορά είναι στ'
# text

## Ας συγκρίνουμε τα ίδια διαμερίσματα

In [31]:
# κρατ΄αμε τα διαμερίσματα με ένα μπάνιο και δύο κρεβάτια
room_specific = ent_house.loc[(ent_house['bathrooms_text'].str.contains('1 ', na=False)) & (ent_house['beds'] == 2)]
len(room_specific)

3433

In [32]:
# επαναλμβάνουμε την ανάλυση
price_per_night_area=room_specific.groupby(by=['neighbourhood_cleansed', 'many_properties'])['cleaned_price'].agg(['median', 'mean']).reset_index()
price_per_night_area

,neighbourhood_cleansed,many_properties,median,mean
0,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,0,50.0,54.428571
1,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,1,79.0,96.600000
2,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,0,45.0,43.625000
3,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,1,54.0,55.000000
4,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,0,49.0,61.326087
5,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,1,49.0,52.683333
6,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,0,44.0,52.285714
7,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,1,58.0,57.266667
8,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,0,45.0,49.105263
9,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,1,47.0,48.200000


In [33]:
room_specific.groupby(by=['neighbourhood_cleansed', 'many_properties'])['cleaned_price'].median().sort_values()

neighbourhood_cleansed            many_properties
ΓΚΥΖΗ                             1                   39.0
ΚΟΛΟΚΥΝΘΟΥ                        1                   40.0
ΑΝΩ ΚΥΨΕΛΗ                        1                   40.0
ΡΙΖΟΥΠΟΛΗ                         0                   41.0
ΝΙΡΒΑΝΑ                           0                   41.0
ΚΟΛΟΚΥΝΘΟΥ                        0                   42.0
ΠΕΝΤΑΓΩΝΟ                         0                   43.5
ΝΙΡΒΑΝΑ                           1                   44.0
ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ                    0                   44.0
ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ                  0                   45.0
ΑΝΩ ΠΑΤΗΣΙΑ                       0                   45.0
ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ                 0                   45.0
ΣΤΑΘΜΟΣ ΛΑΡΙΣΗΣ                   0                   47.0
ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ                 1                   47.0
ΠΕΔΙΟ ΑΡΕΩΣ                       0                   47.5
ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ                   0                   48.0
ΚΟΛΩΝΟ

### Για όλη την Αθήνα

In [34]:
price_per_night_spec=room_specific.groupby(by=['many_properties'])['cleaned_price'].agg(['median', 'mean']).reset_index()

In [35]:
amateur_hosts =price_per_night_spec['median'][0]
prof_hosts = price_per_night_spec['median'][1]

precentage_difference=((prof_hosts-amateur_hosts)/((prof_hosts+amateur_hosts)/2))*100

text= f'Η μέση τιμή του διαμερίσματος ανά μέρα για τους οικοδεσπότες με 3 και πάνω ακίνητα είναι {prof_hosts} \
και για όσους έχουν έχουν κάτω απο 3 είναι {amateur_hosts}. Οι τιμές αυτές έχουν διαφορά {round(precentage_difference)}%'
text

'Η μέση τιμή του διαμερίσματος ανά μέρα για τους οικοδεσπότες με 3 και πάνω ακίνητα είναι 68.0 και για όσους έχουν έχουν κάτω απο 3 είναι 58.0. Οι τιμές αυτές έχουν διαφορά 16%'

### Ποιές είναι οι 5 περιοχές με τη μεγαλύτερη διαφορά τιμών;

In [36]:
pay_spec=room_specific.groupby(by=['neighbourhood_cleansed', 'many_properties'])['cleaned_price'].agg(['median']).reset_index().copy()
pay_spec

,neighbourhood_cleansed,many_properties,median
0,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,0,50.0
1,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,1,79.0
2,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,0,45.0
3,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,1,54.0
4,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,0,49.0
5,ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,1,49.0
6,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,0,44.0
7,ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,1,58.0
8,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,0,45.0
9,ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,1,47.0


In [38]:
pay_comp_spec=pay_spec.pivot_table(values='median', index='neighbourhood_cleansed', columns='many_properties').reset_index().copy()
pay_comp_spec=pay_comp_spec.rename(columns={0:'less_than_3', 
                                  1:'more_than_3'})

In [39]:
pay_comp_spec['price_difference']= (pay_comp_spec.more_than_3 - pay_comp_spec.less_than_3 )

In [40]:
pay_comp_spec = pay_comp_spec.sort_values(by='price_difference', ascending=False).reset_index(drop=True).copy()

In [41]:
areas_greatest_price_dif_spec= ', '.join(pay_comp['neighbourhood_cleansed'].loc[:4].tolist())

text= f' Οι 5 γειτονιές με τη μεγαλύτερη διαφορά τιμών μεταξύ των επαγγελματιών host \
και των ερασιτεχνών για τα δωμάτια με ένα μπάνιο και δύο κρεβάτια είναι οι {areas_greatest_price_dif_spec}'
text

' Οι 5 γειτονιές με τη μεγαλύτερη διαφορά τιμών μεταξύ των επαγγελματιών host και των ερασιτεχνών για τα δωμάτια με ένα μπάνιο και δύο κρεβάτια είναι οι ΠΟΛΥΓΩΝΟ, 1Ο ΝΕΚΡΟΤΑΦΕΙΟ, ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ'

In [42]:
#Σε πόσες περιοχές τα διαμέρίσματα που ανήκουν σε επαγγελματίες οικοδεσπότες είναι πιο ακριβά;
more=[]
less=[]

for i, item in pay_comp_spec.iterrows():
    if item['price_difference'] > 0:
        more.append(item['neighbourhood_cleansed'])
    elif item['price_difference'] < 0:
        less.append(item['neighbourhood_cleansed'])

In [43]:
#  βρίσκουμε την διαφορά για κάθε περιοχή
pay_comp_spec['percentage_difference'] = ((pay_comp_spec['more_than_3'] - pay_comp_spec['less_than_3']) / ((pay_comp_spec['more_than_3'] + pay_comp_spec['less_than_3']) / 2)) * 100
pay_comp_spec=pay_comp_spec.sort_values(by='percentage_difference', ascending=False)

In [44]:
more_than_0= len(more)
less_than_0= len(less)

text= f'Σε {more_than_0} περιοχές της Αθήνας, τα διαμερίσματα που ανήκουν σε επαγγελματίες hosts είναι πιο ακριβά από \
ότι τα διαμερίσματα που ανήκουν σε ερασιτέχνες hosts που είναι ακριβότερα σε {less_than_0} περιοχές.'
text

'Σε 30 περιοχές της Αθήνας, τα διαμερίσματα που ανήκουν σε επαγγελματίες hosts είναι πιο ακριβά από ότι τα διαμερίσματα που ανήκουν σε ερασιτέχνες hosts που είναι ακριβότερα σε 8 περιοχές.'

# Ερωτήσεις

### 1) Πόσοι host έχουν πάνω από δύο καταχωρήσεις;

In [32]:
how_many_porperties= listing['many_properties'].value_counts(normalize=True)*100\

text= f' Στην πλατφόρμα του Airbnb, to {round(how_many_porperties[1])} των host(οικοδεσποτών) έχουν περισσότερα από 2 ακίνητα.'
text

' Στην πλατφόρμα του Airbnb, to 14 των host(οικοδεσποτών) έχουν περισσότερα από 2 ακίνητα.'

### 2) Πόσες καταχη=ωρήσεις ανήκουν σε αυτούς τους host;

In [33]:
owned_by_prof=kk[1]
owned_by_amateurs=kk[0]

text= f'Από τις κατοικίες που έχουν καταχωρηθεί στην πλατφόρμα, το {round(owned_by_prof)}% ανήκει σε επαγγελματίες οικοδεσπότες \
ένω το {round(owned_by_amateurs)}% ανήκει σε οικοδεσπότες χωρίς επαγγελματική δραστηριότητα.'

text

'Από τις κατοικίες που έχουν καταχωρηθεί στην πλατφόρμα, το 54% ανήκει σε επαγγελματίες οικοδεσπότες ένω το 46% ανήκει σε οικοδεσπότες χωρίς επαγγελματική δραστηριότητα.'

 ### 3) Ποια είναι η μέση τιμή ενός διαμερίσματος ανά ημέρα όταν νοικιάζεται από χρήστη που διαθέτει κάτω  από 3 ακίνητα (ερασιτέχνης) και τρία ή περισσότερα ακίνητα? (σε όλη την Αθήνα και ανά περιοχή). 

In [34]:
amateur_hosts = price_per_night['median'][0]
prof_hosts = price_per_night['median'][1]

precentage_difference=((prof_hosts-amateur_hosts)/((prof_hosts+amateur_hosts)/2))*100

text= f'Η μέση τιμή του διαμερίσματος ανά μέρα για τους οικοδεσπότες με 3 και πάνω ακίνητα είναι {prof_hosts} \
και για όσους έχουν έχουν κάτω απο 3 είναι {amateur_hosts}. Οι τιμές αυτές έχουν διαφορά {round(precentage_difference)}%'
text

'Η μέση τιμή του διαμερίσματος ανά μέρα για τους οικοδεσπότες με 3 και πάνω ακίνητα είναι 73.0 και για όσους έχουν έχουν κάτω απο 3 είναι 60.0. Οι τιμές αυτές έχουν διαφορά 20%'

In [44]:
text=f'Για κάθε περιοχή ξεχωριστά:'
text
pay_comp

many_properties,neighbourhood_cleansed,less_than_3,more_than_3,price_difference,percentage_difference
0,ΠΟΛΥΓΩΝΟ,55.0,134.0,79.0,83.597884
1,1Ο ΝΕΚΡΟΤΑΦΕΙΟ,49.0,93.0,44.0,61.971831
4,ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,40.0,73.0,33.0,58.407080
3,ΡΙΖΟΥΠΟΛΗ,59.0,100.0,41.0,51.572327
2,ΖΑΠΠΕΙΟ,76.0,118.5,42.5,43.701799
6,ΠΕΔΙΟ ΑΡΕΩΣ,50.0,72.0,22.0,36.065574
5,ΑΚΡΟΠΟΛΗ,78.5,109.0,30.5,32.533333
10,ΕΛΛΗΝΟΡΩΣΩΝ,53.5,70.0,16.5,26.720648
7,ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ,74.0,96.0,22.0,25.882353
11,ΠΑΓΚΡΑΤΙ,57.0,73.0,16.0,24.615385


### 4) Ποιές είναι οι 5 περιοχές με τη μεγαλύτερη διαφορά μεταξύ αυτών των δύο τιμών. Σε πόσες περιοχές τα διαμερίσματα είναι πιο ακριβά; 

In [36]:
more_than_0= len(more)
less_than_0= len(less)

text= f'Σε {more_than_0} περιοχές της Αθήνας, τα διαμερίσματα που ανήκουν σε επαγγελματίες hosts είναι πιο ακριβά από \
ότι τα διαμερίσματα που ανήκουν σε ερασιτέχνες hosts που είναι ακριβότερα σε {less_than_0} περιοχές.'
text

'Σε 31 περιοχές της Αθήνας, τα διαμερίσματα που ανήκουν σε επαγγελματίες hosts είναι πιο ακριβά από ότι τα διαμερίσματα που ανήκουν σε ερασιτέχνες hosts που είναι ακριβότερα σε 8 περιοχές.'

In [35]:
areas_greatest_price_dif= ', '.join(pay_comp['neighbourhood_cleansed'].loc[:4].tolist())

text= f'Οι 5 γειτονιές με τη μεγαλύτερη διαφορά τιμών μεταξύ των επαγγελματιών host\
και των ερασιτεχνών είναι οι {areas_greatest_price_dif}'
text

'Οι 5 γειτονιές με τη μεγαλύτερη διαφορά τιμών μεταξύ των επαγγελματιών hostκαι των ερασιτεχνών είναι οι ΠΟΛΥΓΩΝΟ, 1Ο ΝΕΚΡΟΤΑΦΕΙΟ, ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ'